In [1]:
import sys
sys.path.append('../src')
from utils import *
import json
import numpy as np
import os
from sklearn.cluster import KMeans


/Users/Pengyun_Wang/anaconda3/envs/HSBC/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
emb_file = '/Users/Pengyun_Wang/Projects/HSBC/data/gte-embeddings.npy'
response_file = '/Users/Pengyun_Wang/Projects/HSBC/data/openai_responses.json'
with open(response_file, 'r') as f:
    response_data = json.load(f)

embeddings = np.load(emb_file)

In [3]:
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

def get_model(n_clusters, embeddings, n_components=50):
    pca = PCA(n_components=n_components)
    kmeans = KMeans(n_clusters=n_clusters)
    pipeline = Pipeline([
        ('scaling', StandardScaler()), 
        ('dimensionality_reduction', pca),
        ('clustering', kmeans)
    ])
    pipeline.fit(embeddings)
    return pipeline

In [3]:
# test the number of clusters
for n_clusters in range(2, 11):
    pipeline = get_model(n_clusters, embeddings)
    labels = pipeline.predict(embeddings)
    silhouette_avg = silhouette_score(embeddings, labels)
    print(f'Number of clusters: {n_clusters}, silhouette score: {silhouette_avg}')

/Users/Pengyun_Wang/anaconda3/envs/HSBC/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [4]:
optimal_n_clusters = 5
pipeline = get_model(optimal_n_clusters, embeddings, n_components=5)
labels = pipeline.predict(embeddings)

/Users/Pengyun_Wang/anaconda3/envs/HSBC/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [32]:
from collections import defaultdict
urls = list(response_data.keys())
clusters = defaultdict(list)
for i, r in enumerate(urls):
    clusters[str(labels[i])].append(r)

# write clusters to file
cluster_file = '/Users/Pengyun_Wang/Projects/HSBC/data/clusters.json'
with open(cluster_file, 'w') as f:
    json.dump(clusters, f, indent=4)